In [8]:
"""
This script is used to check your local docker images.

Two methods can be used - shell commands with subprocess.Popen, or
use the docker SDK (docker-py) to manage and inspect local configuration and running images.

A function is declared to wrap executing a command insider an image which retrieves the contract addresses
from the contract artifact JSON files (ABIs). Use this to confirm that your local configuration (as
specified in config_local.ini) matches the deployed images.
"""


'\nThis script is used to check your local docker images.\n\nTwo methods can be used - shell commands with subprocess.Popen, or\nuse the docker SDK (docker-py) to manage and inspect local configuration and running images.\n\nA function is declared to wrap executing a command insider an image which retrieves the contract addresses\nfrom the contract artifact JSON files (ABIs). Use this to confirm that your local configuration (as\nspecified in config_local.ini) matches the deployed images.\n'

In [1]:
import docker
client = docker.from_env()
import subprocess
# import subprocess
# import pathlib
# import squid_py.ocean as ocean
import sys
from pprint import pprint

In [2]:
import logging

loggers_dict = logging.Logger.manager.loggerDict

logger = logging.getLogger()
logger.handlers = []

# Set level
logger.setLevel(logging.DEBUG)

# FORMAT = "%(asctime)s - %(levelno)s - %(module)-15s - %(funcName)-15s - %(message)s"
# FORMAT = "%(asctime)s %(levelno)s: %(module)30s %(message)s"
FORMAT = "%(levelno)s - %(module)-15s - %(funcName)-15s - %(message)s"

DATE_FMT = "%Y-%m-%d %H:%M:%S"
DATE_FMT = "%Y-%m-%d %H:%M:%S"
formatter = logging.Formatter(FORMAT, DATE_FMT)

# Create handler and assign
handler = logging.StreamHandler(sys.stderr)
handler.setFormatter(formatter)
logger.handlers = [handler]
logger.debug("Logging started")

10 - <ipython-input-2-f3cf088e1dc8> - <module>        - Logging started


In [3]:
# s = subprocess.check_output('docker ps', shell=True).wait()
s = subprocess.Popen("docker ps" + "", shell=True).wait()
print(s)

0


In [4]:
# High level client
client = docker.from_env()
# Get the APIClient for running commands
low_level_api_client = docker.APIClient(base_url='unix://var/run/docker.sock')

10 - config          - find_config_file - Trying paths: ['/home/batman/.docker/config.json', '/home/batman/.dockercfg']
10 - config          - find_config_file - Found file at path: /home/batman/.docker/config.json
10 - auth            - load_config     - Found 'auths' section
10 - auth            - parse_auth      - Found entry (registry='133126667150.dkr.ecr.eu-central-1.amazonaws.com', username='AWS')
10 - auth            - parse_auth      - Found entry (registry='https://index.docker.io/v1/', username='plasmafrog')
10 - config          - find_config_file - Trying paths: ['/home/batman/.docker/config.json', '/home/batman/.dockercfg']
10 - config          - find_config_file - Found file at path: /home/batman/.docker/config.json
10 - auth            - load_config     - Found 'auths' section
10 - auth            - parse_auth      - Found entry (registry='133126667150.dkr.ecr.eu-central-1.amazonaws.com', username='AWS')
10 - auth            - parse_auth      - Found entry (registry='htt

In [6]:
for container in client.containers.list():
#     print(container.name, container.status)
    print(container.image.tags, container.status)
#     print(container.labels)
#     print("\n\n")
# container.logs()

10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/json?limit=-1&all=0&size=0&trunc_cmd=0 HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/66e9f1df7a5f41429d7040b81d58f52f91b37087e6a42b47c2c62dde10fc0414/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/a655a027ed8c127491116206d7dba8b942b50022bd1e2f73b682ff94b8130a57/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/8a2a0ae5c03bff4a17b7655b96a69a5c7d20a258ade93617f0f2da1d9f79202c/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/images/8945c5d267762d7a19ab0eb6eefb18c5a5dd71abef1d464acb873e61fb41abd4/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/images/917474628256d41565da3941d4e45b1b6ebaf2843e0782ee5bbafb365969f5ba/json HTTP/1.1

['oceanprotocol/provider:develop'] running
['oceanprotocol/keeper-contracts:latest'] running
['mongo:3.6'] running


In [14]:
def get_address(api_client, container_id,contract_name):

    # This is the python script to be executed in the running image
    python_script = r"import sys, json; print(json.load(open('/keeper-contracts/artifacts/{}.development.json', 'r'))['address'])".format(contract_name)

    # Wrap the script in quotes (string) and add the python shell command
    command = r"python -c " + '"' + python_script + '"'

    # Create and run the command
    ex = api_client.exec_create(container=container_id, cmd=command)

    return api_client.exec_start(ex)

# Get the docker image running the smart contracts
container_keeper_contracts = client.containers.get('docker_keeper-contracts_1')
addresses=dict()
addresses['market.address'] = get_address(low_level_api_client,container_keeper_contracts.id,'OceanMarket').decode("utf-8").rstrip()
addresses['auth.address'] = get_address(low_level_api_client,container_keeper_contracts.id,'OceanToken').decode("utf-8").rstrip()
addresses['token.address'] = get_address(low_level_api_client,container_keeper_contracts.id,'OceanAuth').decode("utf-8").rstrip()

print("Artifact addresses retrieved:")
pprint(addresses)



10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/docker_keeper-contracts_1/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/containers/a655a027ed8c127491116206d7dba8b942b50022bd1e2f73b682ff94b8130a57/exec HTTP/1.1" 201 74
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/exec/3aacb7821da7fa21e8f487f95e5e73319d42c75fdd6ee1060b83d3a3a783bb5f/start HTTP/1.1" 101 0
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/containers/a655a027ed8c127491116206d7dba8b942b50022bd1e2f73b682ff94b8130a57/exec HTTP/1.1" 201 74
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/exec/803b0617e953db4c9397e62450dbf7371feef67d8649101ad6fb5541f2db1709/start HTTP/1.1" 101 0
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/containers/a655a027ed8c127491116206d7dba8b942b50022bd1e2f73b682ff94b8130a57/exec HTTP/1.1" 201 74
10 - c

Artifact addresses retrieved:
{'auth.address': '0xCfEB869F69431e42cdB54A4F4f105C19C080A601',
 'market.address': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
 'token.address': '0x0E696947A06550DEf604e82C26fd9E493e576337'}
